In [ ]:
import warnings
import numpy as np
import seaborn as sns
import mysql.connector as sql
import gc
import os
import time
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from matplotlib.colors import ListedColormap
import cupy as cp  # Para aprovechar la GPU en cálculos complejos

# Desactivar warnings innecesarios
warnings.filterwarnings('ignore')

inicio = time.time()

### ITERAMOS CADA CELLNAME Y GRAFICAMOS SU FORMA

In [ ]:
# Función corregida para generar la gráfica y guardarla
def generar_grafica(row, df_interf_prb, df_freq_4g):
    df_cell = df_interf_prb[df_interf_prb['CELLNAME'] == row['CELLNAME']]
    if df_cell.empty:
        return

        

    # Forzar uso de CuPy para arrays
    freq_values = df_cell["DLEARFCN"].unique()
    if len(freq_values) == 0:
        return
    FREQ_CELLNAME = freq_values[0]

    # Asegurarnos de obtener un escalar de las columnas
    freq_row = df_freq_4g[df_freq_4g["DLEARFCN"] == FREQ_CELLNAME]
    if freq_row.empty:
        return

    # Extraer los valores como escalares en lugar de arrays
    ini_ul = float(freq_row["frec_ini_ul"].values[0]) + float(freq_row["guarda_izq"].values[0])

    df_cell = df_cell.sort_values('NIVEL_PRB')

    # Forzar el uso de CuPy para arrays en todas las operaciones
    niveles_prb = (df_cell["NIVEL_PRB"] + 1) * 0.18 + ini_ul
    avg_prb_interference = df_cell["L_UL_INTERFERENCE_AVG_PRB"].values

    if len(niveles_prb) != len(avg_prb_interference):
        return

    # Crear el gráfico con subplots
    fig = plt.figure(figsize=(1.0, 1.0))
    gs = GridSpec(2, 1, height_ratios=[3, 1])
    ax0 = fig.add_subplot(gs[0])

    # Definir los colores para el heatmap
    base_colors = [
        (0.0, 0.0, 1.0),
        (0.12549019607843137, 0.6980392156862745, 0.6666666666666666),
        (0.0, 1.0, 0.0),
        (1.0, 1.0, 0.0),
        (1.0, 0.6470588235294118, 0.0),
        (1.0, 0.0, 0.0)
    ]
    colors = np.vstack([np.linspace(base_colors[i], base_colors[i + 1], 40) for i in range(len(base_colors) - 1)])
    cmap = ListedColormap(colors)

    try:
        # Asegurarse de que los datos estén en NumPy para gráficos
        pivot_table = pd.pivot_table(
            df_cell,
            values='L_UL_INTERFERENCE_AVG_PRB',
            index='RESULT_TIME',
            columns='NIVEL_PRB'
        )
        sns.heatmap(data=pivot_table, cmap=cmap, vmin=-120, vmax=-70, cbar=False, ax=ax0)
    except ValueError:
        return
    
    ax0.set_xticks([])
    ax0.set_yticks([])

    ax1 = fig.add_subplot(gs[1])
    ax1.set_xlim([niveles_prb.min(), niveles_prb.max()])
    ax1.plot(niveles_prb, avg_prb_interference, color='blue', linestyle='-', label='Interferencia UL')
    ax1.set_xticks([])
    ax1.set_yticks([])
    ax0.axis('off')
    ax1.axis('off')

    # Guardar la gráfica
    ruta_guardado = os.path.join("D:/Python/TESIS/", "MUESTRAS_V2")
    os.makedirs(ruta_guardado, exist_ok=True)  # Cambiado existiendo por exist_ok
    nombre_archivo = row['CELLNAME'].replace('/', '_').replace(':', '_')
    ruta_guardado_completa = os.path.join(ruta_guardado, f'{nombre_archivo}.png')

    plt.tight_layout()
    plt.savefig(ruta_guardado_completa, bbox_inches='tight', pad_inches=0, dpi=144)
    plt.close(fig)

    # Liberar memoria de la GPU y colectar basura
    cp.get_default_memory_pool().free_all_blocks()  # Liberar memoria de la GPU
    gc.collect()

def procesar_celda(cellname, df_interf_prb, df_freq_4g, conn):
    try:
        row = {'CELLNAME': cellname}
        generar_grafica(row, df_interf_prb, df_freq_4g)
        # Eliminar el registro procesado de la base de datos
        delete_query = "DELETE FROM interf_lte_etl WHERE CELLNAME = %s"
        cursor = conn.cursor()
        cursor.execute(delete_query, (cellname,))
        conn.commit()
    except Exception as e:
        print(f"Error al procesar CELLNAME {cellname}: {e}")

# Conexión a la base de datos
conn = sql.connect(
    host='localhost',
    user='root',
    password='AAAAAAAA',
    database='kpis_4g'
)

# Cargar datos desde la base de datos
query_interf_prb = "SELECT * FROM interf_lte_etl"
df_interf_prb = pd.read_sql(query_interf_prb, conn)

query_freq_4g = "SELECT * FROM freq_4g"
df_freq_4g = pd.read_sql(query_freq_4g, conn)

# Obtener CELLNAME únicos
cellnames = df_interf_prb['CELLNAME'].unique()

# Procesar secuencialmente para pruebas iniciales
for cellname in cellnames:
    procesar_celda(cellname, df_interf_prb, df_freq_4g, conn)

# Cerrar la conexión a la base de datos
conn.close()
print("Proceso completado.")


In [ ]:
fin = time.time()

In [ ]:
total_time = fin - inicio

In [ ]:
total_time